In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale
%config Completer.use_jedi = True

In [2]:
# df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")
# sample = df.iloc[:1000, :].copy()
# sample.to_pickle("sample.pkl")

In [3]:
df = pd.read_pickle("donors.pkl")
sample = pd.read_pickle("sample.pkl")

# EDA DONE

In [4]:
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = sample.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(sample[mailOrderOffers])

for col in groupMailOffers:
    sample[col] = sample[col].fillna(0).astype(np.int16)

In [5]:
searchName = "MDMAUD"
namesToGroup = [col for col in sample.columns.to_list() if searchName in col]
print(namesToGroup)

['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']


In [6]:
for name in namesToGroup:
    setattr(sample, name, sample[name].str.replace("X", "").replace("", np.nan))

In [7]:
# MAJOR
namesToGroup = ["MAJOR"] + namesToGroup
sample.MAJOR.replace({"X": True, " ": False}, inplace=True)

In [8]:
# RFA_2
groupRFA_2 = ["RFA_2", "RFA_2R", "RFA_2F", "RFA_2A"]
namesToGroup += groupRFA_2
sample[namesToGroup]

,MAJOR,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_2,RFA_2R,RFA_2F,RFA_2A
0,False,NaN,NaN,NaN,NaN,L4E,L,4,E
1,False,NaN,NaN,NaN,NaN,L2G,L,2,G
2,False,NaN,NaN,NaN,NaN,L4E,L,4,E
3,False,NaN,NaN,NaN,NaN,L4E,L,4,E
4,False,NaN,NaN,NaN,NaN,L2F,L,2,F
...,...,...,...,...,...,...,...,...,...
995,False,NaN,NaN,NaN,NaN,L2F,L,2,F
996,False,NaN,NaN,NaN,NaN,L1G,L,1,G
997,False,NaN,NaN,NaN,NaN,L1F,L,1,F
998,False,NaN,NaN,NaN,NaN,L4D,L,4,D


In [9]:
# TODO: Create categorical orders
for c in namesToGroup:
    sample[c] = pd.Categorical(sample[c])

In [10]:
sample[namesToGroup].dtypes

MAJOR       category
MDMAUD      category
MDMAUD_R    category
MDMAUD_F    category
MDMAUD_A    category
RFA_2       category
RFA_2R      category
RFA_2F      category
RFA_2A      category
dtype: object

In [11]:
toReorder = ["ADATE", "RFA", "RDATE", "RAMNT"]
reorderedCols = []
for i in range(3, 25):
    reorderedCols += [str(col)+"_"+str(i) for col in toReorder]
reorderedCols = ["ADATE_2"] + groupRFA_2 + reorderedCols

dictIdx = {col: sample.columns.get_loc(col) for col in reorderedCols}

temp = sample[dictIdx.keys()].copy()
sample.drop(columns=dictIdx.keys(), inplace=True)

for key in dictIdx.keys():
    sample[key] = temp[key]

In [12]:
# Converting date columns into datetimes
for c in (dateNames := [c for c in sample.columns.values if ("DATE" in c)]):
    sample[c] = pd.to_datetime(sample[c], infer_datetime_format=True)
    
sample.DOB = pd.to_datetime(sample.DOB)

In [13]:
# Always keep this cell last before EDA WIP
sample.to_pickle("sample_eda_done.pkl")

# EDA WIP

In [14]:
# restart work from here
sample = pd.read_pickle("sample_eda_done.pkl")

In [15]:
# Drop entries where
# df[df.ODATEDW < df.DOB]

In [16]:
df["AGE"] = np.int16(2020 - pd.to_datetime(df.DOB).dt.year)
df.AGE = df.AGE.where(df.AGE != 0, np.nan)

# years passed since first gift
df["loyaltyAge"] = np.int16(2020 - pd.to_datetime(df.ODATEDW).dt.year)

In [17]:
df["AGE"].describe()

count    71529.000000
mean        61.705378
std         16.786761
min          3.000000
25%         49.000000
50%         62.000000
75%         76.000000
max         90.000000
Name: AGE, dtype: float64

In [18]:
df[(df.TCODE > 120) & (df.TCODE < 220)].RAMNTALL

7390     126.0
7546      25.0
11400     87.0
25979     75.0
26620     46.0
42487    347.0
45811    170.0
57962    187.0
62336     57.0
91047    120.0
95105    228.0
Name: RAMNTALL, dtype: float64

In [19]:
df[(df.INCOME.isnull() == False) & (df.WEALTH1.isnull() == False)][["INCOME", "WEALTH1", "WEALTH2"]]

,INCOME,WEALTH1,WEALTH2
1,6.0,9.0,9.0
2,3.0,1.0,1.0
3,1.0,4.0,0.0
4,3.0,2.0,NaN
6,4.0,6.0,3.0
...,...,...,...
95400,4.0,9.0,9.0
95401,6.0,8.0,NaN
95406,6.0,6.0,6.0
95408,7.0,9.0,NaN


In [20]:
# Age groups
# age_groups_labels = ["0-18", "18-30", "30-60", "60+"]
age_groups_labels = ["Minor", "Young worker", "Worker", "Retired"]
age_groups_values = [0, 18, 30, 60, np.inf]

df["ageGroup"] = pd.cut(df.AGE, age_groups_values, labels=age_groups_labels)

# Origin groups [loyalty measured in years]
origin_groups_labels = ["0-1", "1-2", "2-5", "5-10", "10-20", "20+"]
origin_groups_values = [0, 1, 2, 5, 10, 20, np.inf]

df["loyaltyGroup"] = pd.cut(df.loyaltyAge, origin_groups_values, labels=origin_groups_labels)


In [24]:
# Family Aggregate Data - loc[:, "DOMAIN":"WEALTH1"]
# FAD_name

domainCats = [
    letter + str(i) for letter in ["U", "C", "S", "T", "R"] for i in range(1, 5)
]
domainCats.reverse()
# df["FAD_domain"] = pd.Categorical(df.DOMAIN, categories=domainCats, ordered=True)

In [36]:
for i in ["R4", "T4", "S4", "C4"]:
  domainCats.remove(i)
domainCats

['R3',
 'R2',
 'R1',
 'T3',
 'T2',
 'T1',
 'S3',
 'S2',
 'S1',
 'C3',
 'C2',
 'C1',
 'U4',
 'U3',
 'U2',
 'U1']

In [26]:
# Categorical variables
cat_vars = [
    "ageGroup", # category
    "loyaltyGroup", # category
    "NOEXCH", # binary
    "RECINHSE", # binary
    "RECP3", # binary
    "RECPGVG", # binary
    "RECSWEEP", # binary
    "MAJOR", # binary
    "MDMAUD", # category
    "MDMAUD_R", # category
    "MDMAUD_F", # category
    "MDMAUD_A", # category
    "DOMAIN", # category
    "HOMEOWNR", # binary
    ""
]

# Continuous variables
cont_vars = [
    "AGE", # 
    "",
]

# Variables for plots
plot_vars = [
    "STATE", 
    "ZIP", 
    "GEOCODE2", 
    "ODATEDW"
]

# Not used variables
drop_vars = [
    "GEOCODE", 
    "MAILCODE", 
    "DOB"
]

In [22]:
sample.RFA_2

0      L4E
1      L2G
2      L4E
3      L4E
4      L2F
      ... 
995    L2F
996    L1G
997    L1F
998    L4D
999    L1F
Name: RFA_2, Length: 1000, dtype: category
Categories (14, object): ['L1E', 'L1F', 'L1G', 'L2E', ..., 'L4D', 'L4E', 'L4F', 'L4G']